<a href="https://colab.research.google.com/github/Rihan-Fei/Canny-with-3-thresholds-V.S.-HED/blob/main/C3%20HED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy import stats

In [ ]:
#Set root directory and output directory
train_image_dir = 'path to your original images'  # Replace with your root directory path
output_dir = 'path to output'  # Replace with your output directory path
prototxt_path = 'path to the weight file'
model_path = 'path to pretrained model'
train_ground_truth_dir = 'path to ground truth model'
# ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
# load HED
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

In [ ]:
#define evaluate function
def calculate_metrics(ground_truth, detected_edges):
    gt_binary = (ground_truth > 0).astype(int).flatten()
    detected_binary = (detected_edges > 0).astype(int).flatten()

    precision = precision_score(gt_binary, detected_binary)
    recall = recall_score(gt_binary, detected_binary)
    f1 = f1_score(gt_binary, detected_binary)

    return precision, recall, f1


In [ ]:
# Store metrics for all models
all_metrics = {
    'canny1': [],
    'canny2': [],
    'canny3': [],
    'hed': []
}

In [ ]:
for filename in os.listdir(train_image_dir):
    if filename.endswith('.jpg'):
        #load the images
        image_path = os.path.join(train_image_dir, filename)
        image = cv2.imread(image_path)

        #load the ground_truth（MAT）
        gt_filename = filename.replace('.jpg', '.mat')
        gt_data = loadmat(os.path.join(train_ground_truth_dir, gt_filename))

        #Extract real edge maps (Boundaries section)
        ground_truth_list = gt_data['groundTruth'][0][0][0]
        boundaries = ground_truth_list[0][0][1]

        #Ensure the boundary is a binary image
        ground_truth = (boundaries > 0).astype(np.uint8) * 255

        #Ensure the size of the ground_truth and Canny edge maps is consistent
        if ground_truth.shape != image.shape[:2]:
            ground_truth = cv2.resize(ground_truth, (image.shape[1], image.shape[0]))

        #Using Canny edge detection, set three hyperparameters
        #Using different Canny edge detection thresholds
        canny_edges1 = cv2.Canny(image, 50, 150)
        canny_edges2 = cv2.Canny(image, 30, 200)
        canny_edges3 = cv2.Canny(image, 0, 100)
        #Ensure the Canny edge graph is a binary graph
        canny_edges1 = (canny_edges1 > 0).astype(np.uint8) * 255
        canny_edges2 = (canny_edges2 > 0).astype(np.uint8) * 255
        canny_edges3 = (canny_edges3 > 0).astype(np.uint8) * 255

        # Use HED detection
        blob = cv2.dnn.blobFromImage(image, 1.0, (image.shape[1], image.shape[0]), (104.00698793,116.66876762,122.67891434), swapRB=True, crop=False)
        net.setInput(blob)
        hed_edges = net.forward()[0, 0] * 255
        hed_edges = hed_edges.astype(np.uint8)

        #Ensure the size of the HED edge map is consistent
        if hed_edges.shape != image.shape[:2]:
            hed_edges = cv2.resize(hed_edges, (image.shape[1], image.shape[0]))

        # Calculate evaluation
        canny_metrics1 = calculate_metrics(ground_truth, canny_edges1)
        canny_metrics2 = calculate_metrics(ground_truth, canny_edges2)
        canny_metrics3 = calculate_metrics(ground_truth, canny_edges3)
        hed_metrics = calculate_metrics(ground_truth, hed_edges)

        # Store evaluation
        all_metrics['canny1'].append(canny_metrics1)
        all_metrics['canny2'].append(canny_metrics2)
        all_metrics['canny3'].append(canny_metrics3)
        all_metrics['hed'].append(hed_metrics)


        print(f"{filename}: Canny 1 - Precision: {canny_metrics1[0]:.2f}, Recall: {canny_metrics1[1]:.2f}, F1: {canny_metrics1[2]:.2f}")
        print(f"{filename}: Canny 2 - Precision: {canny_metrics2[0]:.2f}, Recall: {canny_metrics2[1]:.2f}, F1: {canny_metrics2[2]:.2f}")
        print(f"{filename}: Canny 3 - Precision: {canny_metrics3[0]:.2f}, Recall: {canny_metrics3[1]:.2f}, F1: {canny_metrics3[2]:.2f}")
        print(f"{filename}: HED - Precision: {hed_metrics[0]:.2f}, Recall: {hed_metrics[1]:.2f}, F1: {hed_metrics[2]:.2f}")
        #Result visualization
        combined = np.hstack((image,
                               cv2.cvtColor(canny_edges1, cv2.COLOR_GRAY2BGR),
                               cv2.cvtColor(canny_edges2, cv2.COLOR_GRAY2BGR),
                               cv2.cvtColor(canny_edges3, cv2.COLOR_GRAY2BGR),
                               cv2.cvtColor(hed_edges, cv2.COLOR_GRAY2BGR)))
        plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Original | Canny 1 | Canny 2 | Canny 3 | HED')
        plt.show()

In [ ]:
#Calculate and output the mean, median of all models
for model in all_metrics:
    metrics_array = np.array(all_metrics[model])
    mean_metrics = np.mean(metrics_array, axis=0)
    median_metrics = np.median(metrics_array, axis=0)

    print(f"{model} - Mean: Precision: {mean_metrics[0]:.2f}, Recall: {mean_metrics[1]:.2f}, F1: {mean_metrics[2]:.2f}")
    print(f"{model} - Median: Precision: {median_metrics[0]:.2f}, Recall: {median_metrics[1]:.2f}, F1: {median_metrics[2]:.2f}")